In [28]:
from dotenv import load_dotenv
from helpers import *
import os

load_dotenv()
coinmarketcap_api_key = os.getenv('COINMARKETCAP_API')

if __name__ == "__main__":
    # print(get_map())
    res = compute_holding(coinmarketcap_api_key, 'portfolio.json', 'EUR')

In [19]:
import helpers
import requests

COINMARKETCAP_API_URL = 'https://pro-api.coinmarketcap.com/v1'
api_key = os.getenv('COINMARKETCAP_API')
portfolio = helpers._load('portfolio.json')
slugs = ','.join(portfolio['data'][e]['slug'] for e in portfolio['data'])

headers = {'X-CMC_PRO_API_KEY': api_key}
payload = {'slug': slugs, 'convert': 'EUR'}

r = requests.get('{}/cryptocurrency/quotes/latest'.format(COINMARKETCAP_API_URL), headers=headers, params=payload)

In [54]:
from helpers import _load

def compute_holding(api_key, portfolio_filename, currency):
    try:
        portfolio = _load(portfolio_filename)
    except FileNotFoundError:
        sys.exit("No {} file found".format(portfolio_filename))

    slugs = ','.join(portfolio['data'][e]['slug'] for e in portfolio['data'])

    headers = {'X-CMC_PRO_API_KEY': api_key}
    payload = {'slug': slugs, 'convert': currency}

    try:
        r = requests.get('{}/cryptocurrency/quotes/latest'.format(COINMARKETCAP_API_URL), headers=headers, params=payload)
    except requests.exceptions.RequestException as e:
        sys.exit("Something went wrong: {}".format(e))
    if r.status_code != 200:
        sys.exit("Something went wrong with your call API: {}\n{}".format(r.status_code, r.json()))
    invested = portfolio['invested']['quantity']
    sum, res, quotes = 0, {}, r.json()['data']
    for cur in quotes:
        hold = portfolio['data'][quotes[cur]['symbol']]
        value = quotes[cur]['quote'][currency]['price'] * hold['quantity']
        sum += value
        res[quotes[cur]['symbol']] = {
            'price': quotes[cur]['quote'][currency]['price'],
            'quantity': hold['quantity'],
            'value': quotes[cur]['quote'][currency]['price'] * hold['quantity'],
            'currency': currency,
        }
    res['total'] = {
        'value': sum,
        'currency': currency,
        'profit': sum - invested,
        'ROI': sum / invested
    }
    return res

In [5]:
import uuid
from google.cloud import firestore
from dotenv import load_dotenv

load_dotenv()
db = firestore.Client()
custom_token = uuid.uuid4()

db.collection(u'test').document(str(custom_token)).set({})

update_time {
  seconds: 1635178790
  nanos: 219731000
}

In [1]:
print(os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))

/home/simon/prog/crypto_aggregator/firebase_sa.json
